In [1]:
# from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd

hunt = pd.read_csv('Data\\Hunt\\huntfile.csv')
khar = pd.read_csv('Data\\Kharchenko\\kharchenkofile.csv')
dias = pd.read_csv('Data\\Dias\\diasfile.csv')
xmatch = pd.read_csv('Data\\Hunt\\xmatchfile.csv')

"""In case you want to use fits files"""
# def datahandler(filename):
#     authorname = filename.split('\\')
#     hdul = fits.open(filename)
#     hdr1 = hdul[0].header #first and second header
#     hdr2 = hdul[1].header
#     data, cols = hdul[1].data, hdul[1].columns
#     metadata = [ [ hdr2[f"TTYPE{i+1}"], hdr2.comments[f"TTYPE{i+1}"] ] for i, j in enumerate(cols.names) ]
#     # mydict[authorname]['hdul'] = hdul
#     return hdul[1].data, hdul[1].columns, metadata

xmatch

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.69600,0.3213,0.04456,0.0,0.663,0.3280
1,0,1636-283,Kharchenko+13,position,0.000298,0.002366,0.003970,0.002366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2MASS-GC01,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2MASS-GC02,Bica+18,position,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2MASS-GC02,Kharchenko+13,"position,hipparcos",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29940,7162,vdBergh_85,Kharchenko+13,position,0.009891,0.024375,0.076088,0.024375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29941,7162,vdBergh_85,Dias+02,position,0.010300,0.025379,0.308965,0.025379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29942,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29943,7163,vdBergh_92,Cantat-Gaudin+20,gaia_dr2,0.032700,0.068013,0.145961,0.068013,0.1545,0.527101,0.03532,0.0000,0.03385,0.0,0.527,0.1757


### Getting Kharchenko numbers
Kharchenko clusters: $n_\mathrm{clusters}=2935 $, $n_\mathrm{detected}= 1513 \newline$ 
Steps:
1. Get Kharchenko from crossmatch table
2. Remove records with Sep (seperation between cluster centres) == NaN
3. Check if crossmatched kharchenko 

In [2]:
kharchenko_xm = xmatch.query('SourceCat == "Kharchenko+13"').dropna(subset=['Sep']) #Get crossmatch table with seperation=NaN removed
kharchenko_names_oc = np.array(khar.query('Type != "g"')['Name'], dtype=object) #get clusters names from kharchenko data with type is not 'g'=globular

IDs_hunt = np.unique(hunt.ID.values)
IDs_xm = np.unique(kharchenko_xm.ID.values)
kharchenko_xm

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.6960,0.32130,0.04456,0.0000,0.663,0.3280
1,0,1636-283,Kharchenko+13,position,0.000298,0.002366,0.003970,0.002366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,0.2413,0.05259,0.11460,0.4058,0.406,0.2442
28,114,ASCC_10,Kharchenko+13,position,0.164400,0.031312,0.310144,0.031312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29937,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29938,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,1.9050,0.45130,0.06676,0.2839,0.451,0.2595
29940,7162,vdBergh_85,Kharchenko+13,position,0.009891,0.024375,0.076088,0.024375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29942,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#remove duplicates
type(kharchenko_xm)
kharchenko_xmnodup = kharchenko_xm.drop_duplicates(subset=['ID'], inplace=False) 
kharchenko_xmnodup

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,pmDESep,pmDESigma,PlxSep,PlxSigma,maxSigma,meanSigma
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,1.6960,0.32130,0.04456,0.0000,0.663,0.3280
26,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,0.2413,0.05259,0.11460,0.4058,0.406,0.2442
29,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,51,ASCC_101,Kharchenko+13,position,0.025590,0.008428,0.051181,0.008428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,5400,ASCC_103,Kharchenko+13,position,0.048860,0.017086,0.174505,0.017086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29934,7160,vdBergh_80,Kharchenko+13,position,0.028140,0.026219,0.216444,0.026219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29937,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29938,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,1.9050,0.45130,0.06676,0.2839,0.451,0.2595
29942,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#sort out the globular clusters in the hunt and kharchenko file

hunt_oc = hunt.loc[hunt['Type'] == 'o']
khar_nonglob = khar.loc[khar['Type'] != 'g']
khar_nonglob


,MWSC,Name,Type,n_Type,RAhour,DEdeg,GLON,GLAT,r0,r1,...,e_rc,rt,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
0,1,Berkeley_58,NaN,NaN,0.0045,60.933,116.750,-1.326,0.025,0.087,...,0.25,13.66,3.31,5.19,1.00,COCD,NaN,99.999,9.99,0
1,2,NGC_7801,NaN,NaN,0.0055,50.727,114.717,-11.331,0.015,0.070,...,0.33,9.93,6.14,2.67,1.44,DIAS,NaN,99.999,9.99,0
2,3,FSR_0459,NaN,NaN,0.0085,59.242,116.446,-2.990,0.018,0.055,...,0.21,7.65,4.63,5.62,3.53,DIAS,irc,99.999,9.99,0
3,4,Stock_18,NaN,NaN,0.0265,64.625,117.617,2.266,0.010,0.050,...,0.03,2.14,0.38,355.57,56.43,DIAS,NaN,99.999,9.99,0
4,5,Berkeley_59,NaN,NaN,0.0373,67.425,118.219,5.001,0.035,0.115,...,0.06,6.51,0.74,153.96,12.16,COCD,NaN,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,3780,Frolov_1,NaN,NaN,23.9570,61.623,116.555,-0.581,0.017,0.040,...,0.16,6.01,4.09,7.87,6.78,COCD,NaN,99.999,9.99,0
3002,3781,NGC_7790,NaN,NaN,23.9730,61.198,116.579,-1.021,0.025,0.060,...,0.30,14.63,3.09,4.32,0.59,COCD,NaN,99.999,9.99,0
3003,3782,NGC_7795,NaN,NaN,23.9606,60.083,116.261,-2.093,0.030,0.037,...,1.04,14.78,5.62,0.49,0.13,DIAS,non,99.999,9.99,0
3004,3783,FSR_0460,NaN,NaN,23.9848,60.694,116.561,-1.532,0.020,0.045,...,0.59,16.85,6.90,0.88,0.28,DIAS,irc,99.999,9.99,0


In [5]:
#merge the two

kharhunt = pd.merge(kharchenko_xmnodup, hunt_oc, on= 'ID', how='inner')

kharhunt

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,...,diffAV50,diffAV84,MOD16,MOD50,MOD84,minClSize,isMerged,isGMMMemb,NXmatches,XmatchType
0,0,1636-283,Kharchenko+13,hipparcos,0.000298,0.002366,0.003970,0.002366,3.4450,0.662721,...,1.471477,1.946265,13.771433,14.001991,14.187698,20,0,0,2,1:m
1,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,...,1.094542,1.414898,8.816640,8.995732,9.120986,80,0,0,4,1:m
2,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,...,0.658193,0.981117,7.588992,7.742329,7.899593,20,0,0,1,1:1
3,51,ASCC_101,Kharchenko+13,position,0.025590,0.008428,0.051181,0.008428,NaN,NaN,...,0.381515,0.556617,7.770802,7.926659,8.100071,80,0,0,2,1:m
4,5400,ASCC_103,Kharchenko+13,position,0.048860,0.017086,0.174505,0.017086,NaN,NaN,...,0.728445,1.050949,8.222772,8.388735,8.557868,80,0,0,3,1:m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,7160,vdBergh_80,Kharchenko+13,position,0.028140,0.026219,0.216444,0.026219,NaN,NaN,...,2.555333,2.900121,9.749496,9.897707,10.063605,20,0,0,1,1:1
1536,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,...,1.345328,1.846767,9.630401,9.778452,9.981219,80,0,0,1,1:1
1537,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,...,2.159117,2.505461,10.931759,11.125109,11.264691,80,0,0,1,1:1
1538,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,...,2.381747,2.710961,10.040602,10.191625,10.338547,40,0,0,1,m:1


In [6]:
#merge on literature names
kharmerge = pd.merge(kharhunt, khar_nonglob, left_on= 'NameLit', right_on='Name', how='inner' )

kharmerge

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,...,e_rc,rt_y,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
0,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,...,0.69,8.95,4.21,2.55,0.55,COCD,NaN,99.999,9.99,0
1,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,...,0.28,3.25,0.49,86.95,24.65,COCD,NaN,99.999,9.99,0
2,51,ASCC_101,Kharchenko+13,position,0.025590,0.008428,0.051181,0.008428,NaN,NaN,...,0.55,4.84,2.31,6.11,1.78,COCD,NaN,99.999,9.99,0
3,5400,ASCC_103,Kharchenko+13,position,0.048860,0.017086,0.174505,0.017086,NaN,NaN,...,0.26,6.90,3.25,8.13,2.60,COCD,NaN,99.999,9.99,0
4,52,ASCC_104,Kharchenko+13,position,0.063640,0.043635,0.159102,0.043635,NaN,NaN,...,1.51,7.74,1.10,15.43,10.04,COCD,NaN,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,7160,vdBergh_80,Kharchenko+13,position,0.028140,0.026219,0.216444,0.026219,NaN,NaN,...,0.21,8.79,3.70,6.41,2.44,DIAS,NaN,99.999,9.99,0
1481,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,...,0.17,7.34,4.74,8.88,5.93,DIAS,NaN,99.999,9.99,0
1482,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,...,0.25,5.00,0.86,6.28,1.22,DIAS,NaN,99.999,9.99,0
1483,7163,vdBergh_92,Kharchenko+13,position,0.089140,0.185433,0.356566,0.185433,NaN,NaN,...,0.31,10.99,4.13,5.54,1.45,COCD,NaN,99.999,9.99,0


In [7]:
#merge on allnames
khar_expanded = kharhunt.assign(synonym=kharhunt['AllNames'].str.split(',')).explode('synonym')
allnamemerge = pd.merge(khar_expanded, khar_nonglob, left_on= 'synonym', right_on='Name', how='inner' )
allnamemerge

,ID,NameLit,SourceCat,TypeSourceCat,Sep,SepTidal,SepTidalLit,SepTidalData,pmRASep,pmRASigma,...,e_rc,rt_y,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
0,114,ASCC_10,Kharchenko+13,hipparcos,0.164400,0.031312,0.310144,0.031312,1.0470,0.274158,...,0.69,8.95,4.21,2.55,0.55,COCD,NaN,99.999,9.99,0
1,50,ASCC_100,Kharchenko+13,position,0.025990,0.008916,0.071212,0.008916,NaN,NaN,...,0.28,3.25,0.49,86.95,24.65,COCD,NaN,99.999,9.99,0
2,51,ASCC_101,Kharchenko+13,position,0.025590,0.008428,0.051181,0.008428,NaN,NaN,...,0.55,4.84,2.31,6.11,1.78,COCD,NaN,99.999,9.99,0
3,5400,ASCC_103,Kharchenko+13,position,0.048860,0.017086,0.174505,0.017086,NaN,NaN,...,0.26,6.90,3.25,8.13,2.60,COCD,NaN,99.999,9.99,0
4,52,ASCC_104,Kharchenko+13,position,0.063640,0.043635,0.159102,0.043635,NaN,NaN,...,1.51,7.74,1.10,15.43,10.04,COCD,NaN,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,7165,vdBergh_130,Kharchenko+13,position,0.073390,0.307565,0.638192,0.307565,NaN,NaN,...,0.27,5.31,2.05,3.87,1.31,DIAS,NaN,99.999,9.99,0
1509,7160,vdBergh_80,Kharchenko+13,position,0.028140,0.026219,0.216444,0.026219,NaN,NaN,...,0.21,8.79,3.70,6.41,2.44,DIAS,NaN,99.999,9.99,0
1510,7161,vdBergh_83,Kharchenko+13,position,0.114700,0.154470,0.955673,0.154470,NaN,NaN,...,0.17,7.34,4.74,8.88,5.93,DIAS,NaN,99.999,9.99,0
1511,7162,vdBergh_85,Kharchenko+13,hipparcos,0.009891,0.024375,0.076088,0.024375,0.2275,0.043188,...,0.25,5.00,0.86,6.28,1.22,DIAS,NaN,99.999,9.99,0


In [16]:
#try and get all 2935 kharchenko clusters
arr = np.asarray(khar_nonglob.Type, dtype='str')

unique_elements, counts = np.unique(arr, return_counts=True)
duplicates = unique_elements[counts > 1]

if duplicates.size > 0:
    concatenated_duplicates = ' '.join(duplicates)
    total_duplicates_count = counts[counts > 1].sum()
    print("Concatenated duplicates:", concatenated_duplicates)
    print("Total number of duplicates:", total_duplicates_count)
else:
    print("No duplicates found.")

Concatenated duplicates: a m n nan r
Total number of duplicates: 2858


In [17]:
khar_open = khar_nonglob.loc[khar['Type'] != 'a']

khar_open

,MWSC,Name,Type,n_Type,RAhour,DEdeg,GLON,GLAT,r0,r1,...,e_rc,rt,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
0,1,Berkeley_58,NaN,NaN,0.0045,60.933,116.750,-1.326,0.025,0.087,...,0.25,13.66,3.31,5.19,1.00,COCD,NaN,99.999,9.99,0
1,2,NGC_7801,NaN,NaN,0.0055,50.727,114.717,-11.331,0.015,0.070,...,0.33,9.93,6.14,2.67,1.44,DIAS,NaN,99.999,9.99,0
2,3,FSR_0459,NaN,NaN,0.0085,59.242,116.446,-2.990,0.018,0.055,...,0.21,7.65,4.63,5.62,3.53,DIAS,irc,99.999,9.99,0
3,4,Stock_18,NaN,NaN,0.0265,64.625,117.617,2.266,0.010,0.050,...,0.03,2.14,0.38,355.57,56.43,DIAS,NaN,99.999,9.99,0
4,5,Berkeley_59,NaN,NaN,0.0373,67.425,118.219,5.001,0.035,0.115,...,0.06,6.51,0.74,153.96,12.16,COCD,NaN,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,3780,Frolov_1,NaN,NaN,23.9570,61.623,116.555,-0.581,0.017,0.040,...,0.16,6.01,4.09,7.87,6.78,COCD,NaN,99.999,9.99,0
3002,3781,NGC_7790,NaN,NaN,23.9730,61.198,116.579,-1.021,0.025,0.060,...,0.30,14.63,3.09,4.32,0.59,COCD,NaN,99.999,9.99,0
3003,3782,NGC_7795,NaN,NaN,23.9606,60.083,116.261,-2.093,0.030,0.037,...,1.04,14.78,5.62,0.49,0.13,DIAS,non,99.999,9.99,0
3004,3783,FSR_0460,NaN,NaN,23.9848,60.694,116.561,-1.532,0.020,0.045,...,0.59,16.85,6.90,0.88,0.28,DIAS,irc,99.999,9.99,0


In [23]:
# Find rows in df1 that are not in df2 based on 'column_name'
result_df = khar_open[~khar_open['Name'].isin(allnamemerge['synonym'])]

result_df

,MWSC,Name,Type,n_Type,RAhour,DEdeg,GLON,GLAT,r0,r1,...,e_rc,rt,e_rt,k,e_k,Src,SType,[Fe/H],e_[Fe/H],o_[Fe/H]
1,2,NGC_7801,NaN,NaN,0.0055,50.727,114.717,-11.331,0.015,0.070,...,0.33,9.93,6.14,2.67,1.44,DIAS,NaN,99.999,9.99,0
2,3,FSR_0459,NaN,NaN,0.0085,59.242,116.446,-2.990,0.018,0.055,...,0.21,7.65,4.63,5.62,3.53,DIAS,irc,99.999,9.99,0
8,9,IRAS_00013+6817,NaN,NaN,0.0664,68.565,118.595,6.091,0.010,0.055,...,0.33,5.69,2.54,1.69,0.61,BIEM,irc,99.999,9.99,0
9,14,FSR_0504,NaN,NaN,0.0970,81.840,121.227,19.116,0.010,0.045,...,0.19,10.62,4.26,5.03,2.46,FPOS,NaN,99.999,9.99,0
10,16,BDB_94,r,c,0.1120,65.629,118.332,3.154,0.015,0.045,...,0.08,4.52,2.52,34.14,26.76,BIEM,irc,99.999,9.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,3774,Harvard_21,NaN,NaN,23.9037,61.740,116.209,-0.386,0.020,0.030,...,0.11,6.01,1.40,11.37,3.40,DIAS,non,99.999,9.99,0
2997,3775,FSR_0440,NaN,NaN,23.9101,50.142,113.682,-11.709,0.015,0.043,...,0.86,16.41,4.65,0.55,0.13,FPOS,NaN,99.999,9.99,0
2998,3776,Czernik_45,NaN,NaN,23.9305,64.620,117.011,2.386,0.010,0.028,...,0.19,6.49,1.18,7.31,1.13,DIAS,dub,99.999,9.99,0
3001,3780,Frolov_1,NaN,NaN,23.9570,61.623,116.555,-0.581,0.017,0.040,...,0.16,6.01,4.09,7.87,6.78,COCD,NaN,99.999,9.99,0
